# Neural Network Inference for Fraud Detection Using FHE
expected RAM usage: 6.2GB  
expected runtime: 14 seconds.

## Introduction
 
This example demonstrates a use case in the finance domain as well as demonstrating encrypted machine learning. We will demonstrate how we can use FHE along with neural networks (NN) to carry out predictions for fraud detection while keeping the data, the NN model and the prediction results encrypted at all times. The neural network and dataset determine fraudulent activities based on anonymized transactions. 

This example showcases how you are able to utilize the processing power of an untrusted environment while preserving the privacy of your sensitive data. The demonstration is split into a privileged client that has access to unencrypted data and models, and an unprivileged server that only performs homomorphic computation in a completely encrypted fashion. The data and the NN model are encrypted in a trusted client environment and then are used to carry out predictions in an untrusted or public environment. Finally, the prediction results return encrypted and can only be decrypted by the data owner in the trusted environment. The concept of providing fully outsourced, but fully encrypted computation to a cloud provider is a major motivating factor in the field of FHE. This use case example shows the capability of the SDK to build such applications.

**NOTE: while the client and server are not literally separated (nor demonstrating true remote cloud computation), the concepts generalize. One can imagine running the trusted code on local environment and the prediction code on a less trusted environment like the cloud. Additionally, we are working on FHE cloud that simplifies a lot of this.**

#### This demo uses the Credit Card Fraud Detection dataset, originally taken from: https://www.kaggle.com/mlg-ulb/creditcardfraud
This dataset contains actual anonymized transactions made by credit card holders from September 2013 and is labeled for transactions being fraudulent or genuine. See references at the bottom of the page.

## Use case

Global credit card fraud is expected to reach $35B by 2025 (Nilson Report, 2020) and since the beginning of the COVID-19 pandemic, 40% of financial services firms saw an increase in fraudulent activity (LIMRA, 2020). As well as volume effects, COVID-19 has worsened the false positive issue for over two-thirds of institutions (69%). A key challenge for many institutions is that significant changes in consumer behavior have often resulted in existing fraud detection systems wrongly identifying legitimate behavior as suspected fraud (Omdia, 2021).

With FHE, you are now able to unlock the value of regulated and sensitive PII data in the context of a less trusted cloud environment by performing AI, machine learning, and data analytic computations without ever having to decrypt. By training your AI models with additional sensitive data, you are able to achieve higher accuracy in fraud detection and reduce the false positive rate while also utilizing the many benefits of cloud computing.

FHE can also help to support a zero trust strategy and can implement strong access control measures by keeping the data, the models that process the data and the results generated encrypted and confidential; only the data owner has access to the private key and has the privilege to decrypt the results.

<br>

## Step 1. Load the existing model and dataset into the trusted environment

In this step we are loading a pre-trained model and a dataset while operating on a trusted client environment. The model and data used in this notebook correspond to a credit card fraud dataset. 

For convenience, the model has been pre-trained and is available in the `data_gen` folder, but you can also experiment with the model you generate yourself. To do that, first run the notebook at `data_gen/fraud_detection_demo.ipynb`, then turn off the boolean flag below, and continue with this notebook.

In [ ]:
import utils 

utils.verify_memory()

load_from_pre_prepared = True

from pathlib import Path
if load_from_pre_prepared:
    INPUT_DIR = Path(utils.get_data_sets_dir()) / 'net_fraud'
else:
    INPUT_DIR = Path('data/net_fraud/')

import json
import utils

X_H5 = INPUT_DIR / 'x_test.h5'
Y_H5 = INPUT_DIR / 'y_test.h5'
MODEL_JSON = str(INPUT_DIR / 'model.json')
MODEL_H5 = str(INPUT_DIR / 'model.h5')

batch_size = 4096
plain_samples, labels = utils.extract_batch_from_files(X_H5, Y_H5, batch_size, 0)

print('Loaded samples of shape',plain_samples.shape)

The following figure illustrates the model being used:

![Model](img/model.png)

<br>

## Step 2. Encrypt the neural network in the trusted environment

The next set of steps include the following:

#### 2.1. Load NN architecture and weights using the FHE library

In [ ]:
import pyhelayers

hyper_params = pyhelayers.PlainModelHyperParams()
neural_net_plain = pyhelayers.NeuralNetPlain()
neural_net_plain.init_from_files(hyper_params, [MODEL_JSON, MODEL_H5])

print('neural_net_plain created and initialized')

# NeuralNetPlain hold a neural network that is not encrypted.
# It can handle fully-connected, convolutional, and mean-pool layers.
# Supported activations: square, and higher-degree polynomials.
# It can also be used to run predictions, and compare it with the encrypted model (e.g. for debugging and testing)

#### 2.2. Compile the plain model

Now we take the plain model and run a process called compilation. This runs internally an Optimizer that finds the best parameters for this model. Not only does the Optimizer find the best parameters for you, but it also gives you estimations on the time it would take to predict using a single core, the precision, the memory, the time it would take to encrypt/decrypt, etc. 

The input to the compilation process are some preferences that we have. In this demo:
* We use HEaaN as the underlying backend. It is somewhat faster and takes less memory in this use case.
* We choose the batch size, how many samples would you provide each time for the inference model to do the classification

This step doesn't yet encrypt the model, but prepares a 'profile' object that we can later use.

In [ ]:
he_run_req = pyhelayers.HeRunRequirements()
# Request a HEaaN context if available, or a SEAL context otherwise
he_run_req.set_he_context_options([pyhelayers.HeContext.create(["HEaaN_CKKS", "SEAL_CKKS"])])

# Largest number we'll be able to safely process under encryption is `2^7=128`.
he_run_req.set_integer_part_precision(7)
# Our numbers are theoretically stored with precision of about 2^-30.
he_run_req.set_fractional_part_precision(30)
# Batch size for NN.
he_run_req.optimize_for_batch_size(batch_size)

profile = pyhelayers.HeModel.compile(neural_net_plain, he_run_req)

batch_size = profile.get_optimal_batch_size()
print(profile.to_string())

print("He profile ready")
print("Batch size: ",batch_size)

#### NOTE

* We specify integer precision of `7` bits. If we exceed this number, the results may be erroneous. But remember, no checks can be made when performing computations under encryption as no information on the data can be exposed.

* We specify a fractional part of `30` bits. This means our numbers are theoretically stored with precision of about `2^-30`. This is misleading however: the ciphertext will inherently add some noise, and it will gradually corrupt more and more bits. The larger number we set here, the more precise our results will be, but at a cost of more expensive computation.

We can specify whatever values we'd like. The optimizer will make sure to notify us if there's no feasible configuration given our requirements and the NN architecture we provide.


#### 2.3. Initialize the context

Here we initialize the FHE library based on the paramaters chosen for us in the profile object.

In [ ]:
# Uncomment this to run over a mockup context
# profile.set_not_secure_mockup()

client_context = pyhelayers.HeModel.create_context(profile)
print('Crypto-library ready',client_context.print_signature())

#### 2.4. Encrypt the NN

The HE profile includes a set of requirements from the library, such as the security level, precision, size of ciphertext, multiplication depth, and some more complicated parameters. They were chosen by the optimizer to provide the best performance for the given task.

In [ ]:
client_nn = pyhelayers.NeuralNet(client_context)
client_nn.encode_encrypt(neural_net_plain, profile)

print('client_nn initialized')

#### 2.5. Encrypt the data samples

To encrypt the data we first create an io processor (iop for short).
The iop object is a lightweight object that knows the model's metadata and can be used to encrypt data for it, and later decrypt the output it sends.

In [ ]:
iop = client_nn.create_io_processor()
client_samples = pyhelayers.EncryptedData(client_context)
iop.encode_encrypt_inputs_for_predict(client_samples, [plain_samples])
print('Batch encrypted')

#### 2.6. Save and send
We save the encrypted model, the context, and the samples in preparation for sending them to the server

In [ ]:
nn_buffer = client_nn.save_to_buffer()
samples_buffer = client_samples.save_to_buffer()

# Save the context. Note that this saves all the HE library information, including the 
# public key, allowing the server to perform HE computations.
# The secret key is not saved here, so the server won't be able to decrypt.
# The secret key is never stored unless explicitly requested by the user using the designated 
# method.
context_buffer = client_context.save_to_buffer()

print('Context, model, and samples saved')

<br>

## Step 3. Perform predictions in the untrusted server using encrypted data and neural network

#### 3.1. Load the neural network, samples and context in the server
We first load all the data sent from the client 

In [ ]:
server_context = pyhelayers.load_he_context(context_buffer)
server_nn = pyhelayers.load_he_model(server_context, nn_buffer)
server_samples = pyhelayers.load_encrypted_data(server_context, samples_buffer)
print('server ready')

#### 3.2. Perform inference in cloud/server using encrypted data and encrypted NN

We can now run the inference of the encrypted data and encrypted NN to obtain encrypted results. This computation does not use the secret key and acts on completely encrypted values.

**NOTE: the data, the NN and the results always remain in encyrpted state, even during computation.**

In [ ]:
server_predictions = pyhelayers.EncryptedData(server_context)
with utils.elapsed_timer('predict', batch_size) as timer:
    server_nn.predict(server_predictions, server_samples)
predictions_buffer = server_predictions.save_to_buffer()

<br>

## Step 4. Decrypt the prediction results in the trusted environment

Now we're back on the trusted side. We can load the predictions:

In [ ]:
client_predictions = pyhelayers.load_encrypted_data(client_context,predictions_buffer)
print('predictions loaded')

Now we can decrypt and decode them. The client's side context also has the secret key, so all objects on the client side can perform decryption.

In [ ]:
plain_predictions = iop.decrypt_decode_output(client_predictions)
print('predictions',plain_predictions)

<br>

## Step 5. Assess the results - precision, recall, F1 score

As this classification problem is a binary one, we will assess the results by comparing the positive and negative classifications with the true labels, also calculating the precision, recall and F1 score.

When running the model in the plain (see `data_gen/fraud_detection_demo.ipynb`), we get the following confusion matrix:  
[[4087 1]  
 [1 &emsp; 7]].  
Comparing the plain results with the confusion matrix reported below shows that the FHE model produces the same results as the plain one.

In [ ]:
utils.assess_results(labels, plain_predictions)

In [ ]:
print("RAM usage:", utils.get_used_ram(), "MB")

<br>

References:

<sub><sup> 1.	Andrea Dal Pozzolo, Olivier Caelen, Reid A. Johnson and Gianluca Bontempi. Calibrating Probability with Undersampling for Unbalanced Classification. In Symposium on Computational Intelligence and Data Mining (CIDM), IEEE, 2015 </sup></sub>

<sub><sup> 2.	Dal Pozzolo, Andrea; Caelen, Olivier; Le Borgne, Yann-Ael; Waterschoot, Serge; Bontempi, Gianluca. Learned lessons in credit card fraud detection from a practitioner perspective, Expert systems with applications,41,10,4915-4928,2014, Pergamon </sup></sub>

<sub><sup> 3.	Dal Pozzolo, Andrea; Boracchi, Giacomo; Caelen, Olivier; Alippi, Cesare; Bontempi, Gianluca. Credit card fraud detection: a realistic modeling and a novel learning strategy, IEEE transactions on neural networks and learning systems,29,8,3784-3797,2018,IEEE </sup></sub>

<sub><sup> 4.	Dal Pozzolo, Andrea Adaptive Machine learning for credit card fraud detection ULB MLG PhD thesis (supervised by G. Bontempi) </sup></sub>

<sub><sup> 5.	Carcillo, Fabrizio; Dal Pozzolo, Andrea; Le Borgne, Yann-Aël; Caelen, Olivier; Mazzer, Yannis; Bontempi, Gianluca. Scarff: a scalable framework for streaming credit card fraud detection with Spark, Information fusion,41, 182-194,2018,Elsevier </sup></sub>

<sub><sup> 6.	Carcillo, Fabrizio; Le Borgne, Yann-Aël; Caelen, Olivier; Bontempi, Gianluca. Streaming active learning strategies for real-life credit card fraud detection: assessment and visualization, International Journal of Data Science and Analytics, 5,4,285-300,2018,Springer International Publishing </sup></sub>

<sub><sup> 7.	Bertrand Lebichot, Yann-Aël Le Borgne, Liyun He, Frederic Oblé, Gianluca Bontempi Deep-Learning Domain Adaptation Techniques for Credit Cards Fraud Detection, INNSBDDL 2019: Recent Advances in Big Data and Deep Learning, pp 78-88, 2019 </sup></sub>

<sub><sup> 8.	Fabrizio Carcillo, Yann-Aël Le Borgne, Olivier Caelen, Frederic Oblé, Gianluca Bontempi Combining Unsupervised and Supervised Learning in Credit Card Fraud Detection Information Sciences, 2019 </sup></sub>

<sub><sup> 9.	Yann-Aël Le Borgne, Gianluca Bontempi Machine Learning for Credit Card Fraud Detection - Practical Handbook </sup></sub>